## Imports

In [ ]:
import bokeh.plotting as bpl
import cv2
import holoviews as hv
from IPython import get_ipython
import matplotlib.pyplot as plt
import numpy as np
import os

try:
    cv2.setNumThreads(0)
except():
    pass

try:
    if __IPYTHON__:
        get_ipython().run_line_magic('load_ext', 'autoreload')
        get_ipython().run_line_magic('autoreload', '2')
except NameError:
    pass

import caiman as cm
from caiman.source_extraction.cnmf import cnmf

bpl.output_notebook()
hv.notebook_extension('bokeh')

## Load saved results

In [ ]:
results_dir = "" # Place path to results here

cnmf_fit = cnmf.load_CNMF(os.path.join(results_dir, "cnmfe_results.hdf5"),
                            n_processes=1, 
                            dview=None)
correlation_image = cnmf_fit.estimates.Cn

Yr, dims, num_frames = cm.load_memmap(cnmf_fit.mmap_file)
images = np.reshape(Yr.T, [num_frames] + list(dims), order="F")

motion_corrected = cm.load(os.path.join(results_dir, "motion_correction_comparison.avi"))
print(f"Successfully loaded data.")

## Visualizing results
We've sorted our components into accepted/rejected, so it's time to start looking at the results! Caiman provides many built-in `estimates` methods for visualizaing results. 

In [ ]:
# Play motion corrected video
motion_corrected.play(fr=25, magnification=2, offset=0)

### All contours
We have already used `plot_contours_nb()`, but if we provide it the `idx` keyword it will split the view into accepted and rejected components.

In [ ]:
cnmf_fit.estimates.plot_contours_nb(img=correlation_image, 
                                      idx=cnmf_fit.estimates.idx_components);

### View individual spatial/temporal components
One of the most useful visualization tools is `nb_view_components()`, which lets you scroll through individiual spatial and temporal components. This tool also displays the values of the three evaluation criteria for each component, which can be useful if you feel you need to change your evaluation criteria and re-run `evaluate_components()`. Perhaps you have too many false negatives and want to lower your SNR threshold. 

In [ ]:
# view accepted components
cnmf_fit.estimates.nb_view_components(img=correlation_image, 
                                        idx=cnmf_fit.estimates.idx_components[::4],
                                        cmap='gray',
                                        denoised_color='red',
                                    );

The above shows the raw traces (`C+YrA`) by default, but you can superimpose the denoised traces from `C` if you add a color to the `denoised_color` parameter. As always in Jupyter, if you are unsure how a method works, you can enter `cnmf_refit.estimates.nb_view_components?` in a new cell to get the documentation for the method. 

We can also view the rejected compoonents:

In [ ]:
# rejected components
if len(cnmf_fit.estimates.idx_components_bad) > 0:
    cnmf_fit.estimates.nb_view_components(
        img=correlation_image, 
        idx=cnmf_fit.estimates.idx_components_bad[::8], 
        cmap='gray',
        denoised_color='red',
    )
else:
    print("No components were rejected.")

> One legacy from Matlab with these plotters is that they use one-based indexing when showing `Neuron number`. This can make it confusing when comparing to your own plotting results which will use zero-based indexing.

### Building your own visualizations
> This is a slightly more advanced section that you can safely skip your first time through. 

There are many custom visualizations you can build yourself based on the estimates generated from Caiman. For example, what if you wanted to plot the spatial footprint of a neuron with the contour superimposed?  The contours of the spatial footprints are in `estimates.coordinates`, which is a list of dictionaries corresponding to each component. The dictionary includes a `coordinates` field that contains the x,y coordinates of the contour. Here we'll show how to plot this contour superimposed on the corresponding spatial footprint in `A`. 

The following extracts all of the contours of the accepted components into a list from the estimates (it puts them in a list because they are not all the same size):

In [ ]:
idx_accepted = cnmf_fit.estimates.idx_components
all_contour_coords = [cnmf_fit.estimates.coordinates[idx]['coordinates'] for idx in idx_accepted]

Each footprint in `A` is stored as a compressed sparse column array. We can convert it to a dense array with `toarray()`, and plot the contour and footprint together:

In [ ]:
idx_to_plot = 30
component_number = idx_accepted[idx_to_plot]
component_contour = all_contour_coords[idx_to_plot]
component_footprint = np.reshape(
    cnmf_fit.estimates.A[:, component_number].toarray(),
    dims,
    order='F',
)

In [ ]:
plt.figure(); 
plt.imshow(component_footprint, cmap='gray');
plt.plot(component_contour[:, 0], 
         component_contour[:, 1], 
         color='pink', 
         linewidth=2)
plt.title(f'Footprint/Contour {component_number}');

# A few final things
We have extracted the calcium traces `C`, spatial footprints `A`, and estimated spike counts `S`, which is the main goal with CNMF. But there are a few important things remaining.

## Extract $\Delta F/F$ values
So far our calcium traces are in arbitrary units. It is more common to report the calcium fluorescence relative to some baseline value $F_0$:

$$
\Delta F/F = (F(t)-F_0)/F_0
$$

Traditionally, the baseline value was calculated during some initial period of queiscience (e.g., in the visual system, when the animal was sitting in the dark). However, it is problematic to make any assumptions about a "quiet" baseline period, so researchers have started to use a moving average to calculate $F_0$. This is also what Caiman does.

More specifically, in Caiman the baseline is a running percentile calculated over a `frames_window` moving window. You can calculate $\Delta F/F$ using raw traces or the denoised traces in C (this is toggled using the `use_residuals` argument).

We can plot a temporal trace of a dff with time. In the following, we'll generate a `frame_time` variable for plotting. This is an  idealized time representation using `linspace()`, which assumes that images were acquired at a constant frequency throughout the session. In practice your hardware probably returns the time at which each frame was acquired, so you can use that information in your own analysis if you have it. 

In [ ]:
# plot F_dff
idx_to_plot = 30
idx_accepted = cnmf_fit.estimates.idx_components
component_number = idx_accepted[idx_to_plot]
f, ax = plt.subplots(figsize=(7,2))
ax.plot(np.arange(cnmf_fit.estimates.F_dff.shape[1]) / 10, 
        cnmf_fit.estimates.F_dff[component_number, :], 
        linewidth=0.5,
        color='k');
ax.set_xlabel('Time (s)')
ax.set_ylabel('$\Delta F/F$')
ax.set_title(f"$\Delta F/F$ for unit {component_number}");
plt.tight_layout()

<div class="alert alert-info" markdown="1">
    <h2 style="margin-top: 0;">More on $\Delta F/F$</h2>  
The above discussion of dff leaves out some details about it is actually calculated in Caiman. In practice, Caiman locally projects the model of the background activity to the spatial footprint of the neuron, and adds a moving percentile of this projected trace to the denominator as an additional normalizing term (see the discussion on the background model below). This acts as an empirical fudge factor that can keep $\Delta F/F$ from getting too large for very small values of $F_0$ (especially when using denoised traces). Users may prefer to use the more traditional equation directly. Thanks to Peter Rupprecht for a helpful discussion of this topic. 
</div>